### Set the OPEN AI KEY

In [1]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-None-UYrgNTFk4TjYLNQ0dnX6T3BlbkFJTDopdRybVwanAaD7AaW8'

### Create the LLM model

#### I'm using the chat GPT 3.5 turbo for this task because the other small and free models tested can't execute calls to tools. 

In [2]:
from langchain_openai import ChatOpenAI

# Create the language model
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

### Create the tools that are going to be used by the agent

In [3]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate

class SearchLeagueOfLegendsInput(BaseModel):
    query: str = Field(description="user query about league of legends")

class SearchWorldOfWarcraftInput(BaseModel):
    query: str = Field(description="user query about world of warcraft")

@tool("search-league-of-legends-tool", args_schema=SearchLeagueOfLegendsInput)
def search_league_of_legends_info(query: str) -> str:
    """Look up things about league of legends news and patch notes"""

    llm_lol = ChatOpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio", temperature=0)
    
    messages = [
        SystemMessage(content="You are very powerful assistant called that answers questions exclusively about league of legends."
                "Any other questions should be answered with 'I don't have info about this rigth now, but I'm working on it'."),
        HumanMessage(content=query),
    ]
    output = llm_lol.invoke(messages)

    return output

@tool("search-world-of-warcraft-tool", args_schema=SearchWorldOfWarcraftInput)
def search_world_of_warcraft_info(query: str) -> str:
    """Look up things about world of warcraft news and patch notes"""
    
    llm_wow = ChatOpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio", temperature=0)
    
    messages = [
        SystemMessage(content="You are very powerful assistant that answers questions exclusively about world of warcraft."
                "Any other questions should be answered with 'I don't have info about this rigth now, but I'm working on it'."),
        HumanMessage(content=query),
    ]
    output = llm_wow.invoke(messages)

    return output

tools = [
    search_league_of_legends_info,
    search_world_of_warcraft_info
]

### Create the base prompt that specifies the agent behavior and ensures the tools will be used.

In [4]:

# You are very powerful assistant called Lyra that helps with questions about exclusively about league of legends and world of warcraft.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """            
            You are a very powerful assistant called Lyra that answers questions about league of legends and world of warcraft. 
            Make sure to use your tools for information.

            Key tools:
                - search_league_of_legends_info: use to access up-to-date league of legends data.
                - search_world_of_warcraft_info: use to access up-to-date world of warcraft data.

            Always pass user's question as an argument.
            """,
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### Make request and check the results

In [5]:
output = agent_executor.invoke({"input": "Hello, introduce yourself to someone opening this program for the first time. Be concise.?"})
output["output"]



> Entering new AgentExecutor chain...
Hello! I'm Lyra, your assistant for all things League of Legends and World of Warcraft. I can provide you with up-to-date information, news, and patch notes for both games. Feel free to ask me anything about these games, and I'll be happy to help!

> Finished chain.


"Hello! I'm Lyra, your assistant for all things League of Legends and World of Warcraft. I can provide you with up-to-date information, news, and patch notes for both games. Feel free to ask me anything about these games, and I'll be happy to help!"

In [6]:
output = agent_executor.invoke({"input": "how's is volibear jungling?"})
output["output"]



> Entering new AgentExecutor chain...

Invoking: `search-league-of-legends-tool` with `{'query': 'volibear jungle'}`


content="Volibear is a popular jungler in League of Legends! Here are some key points to consider:\n\n**Pros:**\n\n1. **Tanky:** Volibear has a lot of health and armor, making him a great tanky jungler.\n2. **Strong early game:** His abilities scale well with levels, and his Q (Growl) is particularly strong in the early game.\n3. **Crowd control:** Volibear's W (Frenzy) can lock down enemies, while his E (Ravenous Frenzy) provides a nice slow.\n4. **Sustain:** His ultimate ability, (Sky Splitter), allows him to heal and deal damage to enemies.\n\n**Cons:**\n\n1. **Limited mobility:** Volibear's movement speed is relatively low, making it difficult for him to chase or escape quickly.\n2. **Dependence on Q:** While his W and E are strong, his Q is essential for dealing damage and setting up ambushes.\n3. **Vulnerable in team fights:** Without a reliable way to initiate

"Volibear is a popular jungler in League of Legends! Here are some key points to consider:\n\nPros:\n1. Tanky: Volibear has a lot of health and armor, making him a great tanky jungler.\n2. Strong early game: His abilities scale well with levels, and his Q (Growl) is particularly strong in the early game.\n3. Crowd control: Volibear's W (Frenzy) can lock down enemies, while his E (Ravenous Frenzy) provides a nice slow.\n4. Sustain: His ultimate ability, (Sky Splitter), allows him to heal and deal damage to enemies.\n\nCons:\n1. Limited mobility: Volibear's movement speed is relatively low, making it difficult for him to chase or escape quickly.\n2. Dependence on Q: While his W and E are strong, his Q is essential for dealing damage and setting up ambushes.\n3. Vulnerable in team fights: Without a reliable way to initiate or disengage from team fights, Volibear can be vulnerable to all-ins.\n\nTips and Tricks:\n1. Focus on last-hitting minions: This will help you farm efficiently and sca